In [1]:
def parse_popmap(popmap, individual_dict):
    """
    The function parses a text (i.e. a list of tags)
    and puts the content of the first column of the 
    into a list"""
    
    import gzip
#    from collections import defaultdict
    
#    pop_dict = defaultdict(dict)
    
    if popmap.endswith('.gz'):
        FH = gzip.open(popmap,'rb')
    else:
        FH = open(popmap,'r')
        
    for line in FH:
        line = line.strip()
        per_line = line.split('\t')
        individual_dict[per_line[0]]['population'] = per_line[1]
    
    FH.close()
    
#    return pop_dict

In [2]:
from collections import defaultdict

ind_dict = defaultdict(dict)

parse_popmap(popmap='../popmap', individual_dict=ind_dict)
print sorted(ind_dict)

['sample_BBNB1', 'sample_BBNB10', 'sample_BBNB2', 'sample_BBNB3', 'sample_BBNB4', 'sample_BBNB5', 'sample_BBNB6', 'sample_BBNB7', 'sample_BBNB8', 'sample_BBNB9', 'sample_LBPNK2', 'sample_LBPNK3', 'sample_LBPNK4', 'sample_LBPNK5', 'sample_LBPNK6', 'sample_LBPNK7', 'sample_MONB11', 'sample_MONB2', 'sample_MONB3', 'sample_MONB4', 'sample_MONB5', 'sample_MONB7', 'sample_MONB8', 'sample_MONB9', 'sample_NJ1', 'sample_NJ10', 'sample_NJ2', 'sample_NJ3', 'sample_NJ4', 'sample_NJ5', 'sample_NJ6', 'sample_NJ7', 'sample_NJ8', 'sample_NJ9']


In [3]:
def parse_taglist(tagfile):
    """
    The function parses a text (i.e. a list of tags)
    and puts the content of the first column of the 
    into a list"""
    
    import gzip
    taglistt = []
    
    if tagfile.endswith('.gz'):
        FH = gzip.open(tagfile,'rb')
    else:
        FH = open(tagfile,'r')
        
    for line in FH:
        line = line.strip()
        per_line = line.split('\t')
        taglistt.append(per_line[0])
    
    FH.close()
    
    return taglistt
        

In [53]:
taglist = []
taglist = parse_taglist(tagfile = '../whitelist.txt')
#print taglist

parse catalog
optional keep only relevant tags

In [54]:
def parse_catalog(catalog, tag_whitelist=[]):
    """
    The function parses a stacks catalog
    """
    import gzip
    from collections import defaultdict
    
    catalog_dict_local = defaultdict(list)
    whitelist = []
    
    if tag_whitelist:
        print "You have provided a whitelist of tags\n"
        whitelist = tag_whitelist[:]
    
    if catalog.endswith('.gz'):
        print "open gzipped catalog\n"
        FH_c = gzip.open(catalog,'rb')
    else:
        FH_c = open(catalog,'r')

#    for i in range(100):
#        line = FH_c.readline()
    for line in FH_c:
        line = line.strip()
        per_line = line.split('\t')
        if not tag_whitelist:
            catalog_dict_local[per_line[2]] = per_line[8].split(',')
        else:
            if whitelist:
                if per_line[2] in whitelist:
                    catalog_dict_local[int(per_line[2])] = per_line[8].split(',')
                    whitelist.remove(per_line[2])
                
            else:
                print "Done parsing all relevant tags (n=%i)\n" %len(tag_whitelist)
                break
                
    FH_c.close()
    
    if tag_whitelist:
        print "Parsed %i of %i desired tags\n" %(len(catalog_dict_local), len(tag_whitelist))
    else:
        print "Parsed %i tags\n" %(len(catalog_dict_local))

#    print sorted(catalog_dict)
    
    return catalog_dict_local
        
    

In [55]:
#parse full catalog
catalog_dict = {}
catalog_dict = parse_catalog(catalog='../../../../stacks/ALL/mapping/excl_PCR_dupl/BWA-8MM/M_zebra/2-cstacks/m5/n0/data/batch_1.catalog.tags.tsv.gz')


open gzipped catalog

Parsed 74001 tags



In [56]:
#parse catalog using a whitelist
catalog_dict= {}
#test_list = ['1','2','3','9']
catalog_dict = parse_catalog(catalog='../../../../stacks/ALL/mapping/excl_PCR_dupl/BWA-8MM/M_zebra/2-cstacks/m5/n0/data/batch_1.catalog.tags.tsv.gz', tag_whitelist=taglist)
#print sorted(catalog_dict)


You have provided a whitelist of tags

open gzipped catalog

Done parsing all relevant tags (n=11786)

Parsed 11786 of 11786 desired tags



find stacks synonyms for sample names

In [57]:
def find_stacks_ids(ind_list, stacks_dir, individual_dict):
    """
    The function finds the internal stacks ids for a list of sample names
    """
    
    import os.path
    import glob
    import gzip

    if type(ind_list) is list: #check the value passed through ind_list is already a list
        print "IDs for individuals provided in list"
        sample_list = ind_list[:]
    else: #if it's not a list check if it's a file and parse the first column
        if not os.path.isfile(ind_list):
            IOError('ind_list needs to be either a txt file or an existing list')
        print "parsing IDs for individuals from file"
        sample_list = []
        sample_list = parse_taglist(ind_list)

    
    file_list = glob.glob(stacks_dir+'*tags*')
    if len(file_list) == 0:
        IOError('no tags files detected in the directory that you have specified')
#    print file_list

    #check if there is a file for all samples in the list
    for sample in sorted(sample_list):
#        print sample
        found_tags_file = ""
        if file_list:
            for tags_file in sorted(file_list):
                if sample in tags_file:
#                    print tags_file
                    found_tags_file = tags_file
                    file_list.remove(tags_file)
                    break
        
        if not found_tags_file:#individual_dict[sample].has_key('tags_file'):
            raise IOError("did not find a tags file for sample %s" %sample)
        else:
            individual_dict[sample]['tags_file'] = os.path.abspath(found_tags_file)
#            print individual_dict[sample]['tags_file']
            if found_tags_file.endswith('.gz'):
                tags_FH = gzip.open(found_tags_file, 'rb')
            else:
                tags_FH = open(found_tags_file, 'r')
            
            ID = tags_FH.readline().split("\t")[1]
            tags_FH.close()
            individual_dict[sample]['catalog_id'] = ID
            print "%s -> id: %s" %(sample,ID)
            

In [58]:
find_stacks_ids(ind_list=ind_dict.keys(), individual_dict=ind_dict, stacks_dir='../../../../stacks/ALL/mapping/excl_PCR_dupl/BWA-8MM/M_zebra/1-pstacks/m5/') #ind_dict.keys(), stacks_dir='../stacks_data/pstacks/')

IDs for individuals provided in list
sample_BBNB1 -> id: 149
sample_BBNB10 -> id: 148
sample_BBNB2 -> id: 150
sample_BBNB3 -> id: 151
sample_BBNB4 -> id: 152
sample_BBNB5 -> id: 153
sample_BBNB6 -> id: 154
sample_BBNB7 -> id: 155
sample_BBNB8 -> id: 156
sample_BBNB9 -> id: 157
sample_LBPNK2 -> id: 180
sample_LBPNK3 -> id: 181
sample_LBPNK4 -> id: 182
sample_LBPNK5 -> id: 183
sample_LBPNK6 -> id: 184
sample_LBPNK7 -> id: 185
sample_MONB11 -> id: 209
sample_MONB2 -> id: 210
sample_MONB3 -> id: 211
sample_MONB4 -> id: 212
sample_MONB5 -> id: 213
sample_MONB7 -> id: 214
sample_MONB8 -> id: 215
sample_MONB9 -> id: 216
sample_NJ1 -> id: 227
sample_NJ10 -> id: 226
sample_NJ2 -> id: 228
sample_NJ3 -> id: 229
sample_NJ4 -> id: 230
sample_NJ5 -> id: 231
sample_NJ6 -> id: 232
sample_NJ7 -> id: 233
sample_NJ8 -> id: 234
sample_NJ9 -> id: 235


In [60]:
def find_tag_ids_per_individual(ind_list, individual_dict, catalog, catalog_tag_whitelist=[]):
    """
    The function finds the read ids for all relevant
    tags
    """
    
    import os.path
    import gzip
    
    catalog_ind_ids = {}
    
    if type(ind_list) is list: #check the value passed through ind_list is already a list
        print "IDs for individuals provided in list"
        sample_list = ind_list[:]
    else: #if it's not a list check if it's a file and parse the first column
        if not os.path.isfile(ind_list):
            IOError('ind_list needs to be either a txt file or an existing list')
        print "parsing IDs for individuals from file"
        sample_list = []
        sample_list = parse_taglist(ind_list)

    for ind in sample_list:
        if not individual_dict.has_key(ind):
            raise IOError('no data found for %s' %ind)
        else:
            print "Found data for %s" %ind
            catalog_ind_ids[ind_dict[ind]['catalog_id']] = ind
            ind_dict[ind]['tags'] = {}
    
    if not catalog_tag_whitelist:
        catalog_tag_whitelist = [int(i) for i in sorted(catalog.keys()[:])] #sorted(catalog.keys()[:])
    
    print "Screening through %i tag ids" %len(catalog_tag_whitelist)
    
    for tag in catalog_tag_whitelist:
#        print str(tag)+":"
        for s in catalog[tag]:
            split = s.split("_")
            if catalog_ind_ids.has_key(split[0]):
                ind_dict[catalog_ind_ids[split[0]]]['tags'][int(tag)] = int(split[1]) #[int(split[1])] = int(tag)



In [61]:
find_tag_ids_per_individual(ind_list=ind_dict.keys(), individual_dict=ind_dict, catalog=catalog_dict)

IDs for individuals provided in list
Found data for sample_LBPNK3
Found data for sample_LBPNK2
Found data for sample_LBPNK5
Found data for sample_LBPNK4
Found data for sample_LBPNK7
Found data for sample_MONB11
Found data for sample_BBNB7
Found data for sample_BBNB6
Found data for sample_BBNB5
Found data for sample_BBNB4
Found data for sample_BBNB3
Found data for sample_BBNB2
Found data for sample_BBNB1
Found data for sample_LBPNK6
Found data for sample_BBNB9
Found data for sample_BBNB8
Found data for sample_MONB3
Found data for sample_MONB2
Found data for sample_MONB7
Found data for sample_MONB5
Found data for sample_MONB4
Found data for sample_MONB9
Found data for sample_MONB8
Found data for sample_BBNB10
Found data for sample_NJ10
Found data for sample_NJ9
Found data for sample_NJ8
Found data for sample_NJ1
Found data for sample_NJ3
Found data for sample_NJ2
Found data for sample_NJ5
Found data for sample_NJ4
Found data for sample_NJ7
Found data for sample_NJ6
Screening through 1178

In [62]:
print ind_dict.keys()
for samp in sorted(ind_dict.keys()):
    print samp
    print len(ind_dict[samp]['tags'])
#    print len(ind_dict[samp]['read_ids'].keys())

['sample_LBPNK3', 'sample_LBPNK2', 'sample_LBPNK5', 'sample_LBPNK4', 'sample_LBPNK7', 'sample_MONB11', 'sample_BBNB7', 'sample_BBNB6', 'sample_BBNB5', 'sample_BBNB4', 'sample_BBNB3', 'sample_BBNB2', 'sample_BBNB1', 'sample_LBPNK6', 'sample_BBNB9', 'sample_BBNB8', 'sample_MONB3', 'sample_MONB2', 'sample_MONB7', 'sample_MONB5', 'sample_MONB4', 'sample_MONB9', 'sample_MONB8', 'sample_BBNB10', 'sample_NJ10', 'sample_NJ9', 'sample_NJ8', 'sample_NJ1', 'sample_NJ3', 'sample_NJ2', 'sample_NJ5', 'sample_NJ4', 'sample_NJ7', 'sample_NJ6']
sample_BBNB1
11747
sample_BBNB10
11722
sample_BBNB2
11286
sample_BBNB3
11770
sample_BBNB4
11763
sample_BBNB5
11764
sample_BBNB6
11698
sample_BBNB7
11784
sample_BBNB8
11768
sample_BBNB9
11767
sample_LBPNK2
11779
sample_LBPNK3
11388
sample_LBPNK4
10810
sample_LBPNK5
11725
sample_LBPNK6
11781
sample_LBPNK7
11773
sample_MONB11
11768
sample_MONB2
11776
sample_MONB3
10884
sample_MONB4
11776
sample_MONB5
11215
sample_MONB7
11393
sample_MONB8
11772
sample_MONB9
11446
sa

In [63]:
def find_reads_per_individual(ind_list, individual_dict, catalog, catalog_tag_whitelist=[]):
    """
    The function finds read ids for each individual for each tag
    """
    import os.path
    import gzip
    
    catalog_ind_ids = {}
    
    if type(ind_list) is list: #check the value passed through ind_list is already a list
        print "IDs for individuals provided in list"
        sample_list = ind_list[:]
    else: #if it's not a list check if it's a file and parse the first column
        if not os.path.isfile(ind_list):
            IOError('ind_list needs to be either a txt file or an existing list')
        print "parsing IDs for individuals from file"
        sample_list = []
        sample_list = parse_taglist(ind_list)

    if not catalog_tag_whitelist:
        catalog_tag_whitelist = [int(i) for i in sorted(catalog.keys()[:])]
    
    for ind in sample_list:
        if not individual_dict.has_key(ind):
            raise IOError('no data found for %s' %ind)
        else:
            print "Found data for %s" %ind
            catalog_ind_ids[ind_dict[ind]['catalog_id']] = ind
#            ind_dict[ind]['tags'] = {}
    
    print "Screening through %i tag ids" %len(catalog_tag_whitelist)
    
    for ind in sorted(sample_list):
        print "#################\n"
        print ind
#        print ind_dict[ind]['tags']
#        print sorted(ind_dict[ind]['tags'])
        ind_catalog_tags_to_process = []
        

#        print ind_dict[ind]['tags'].keys()

        for catalog_tag in catalog_tag_whitelist:
            if ind_dict[ind]['tags'].has_key(catalog_tag):
                ind_catalog_tags_to_process.append(ind_dict[ind]['tags'][catalog_tag])
        
        if not ind_catalog_tags_to_process: #if this individual contains any relevant tags
            print "nothing to process for this indidividual"
            continue #go to next individual
            
        print "%s individual tags to process:\n" %len(ind_catalog_tags_to_process)
        ind_dict[ind]['read_ids'] = {}
        for to_process in ind_catalog_tags_to_process:
            ind_dict[ind]['read_ids'][int(to_process)] = []
            
        #open file
        if ind_dict[ind]['tags_file'].endswith('.gz'):
            ind_tags_FH = gzip.open(ind_dict[ind]['tags_file'], 'rb')
        else:
            ind_tags_FH = open(ind_dict[ind]['tags_file'], 'r')
            
        add = False 

        for line in [line.strip() for line in ind_tags_FH]:
            if not ind_catalog_tags_to_process: #if there is tags left to be processed
                break

            if 'consensus' in line:
                if add: #that means that the previous tag was relevant
#                    print "done with tag %s - found %s reads - will be removed from list" %(str(t), len(ind_dict[ind]['read_ids'][int(t)]))
                    ind_catalog_tags_to_process.remove(int(t))
#                    print "still to process:\n%s" %ind_catalog_tags_to_process

                add = False #per default a given tag is not to be added
                t = line.split("\t")[2]
#                print "\n\nFound consensus for tag: %s" %t
                if int(t) in ind_catalog_tags_to_process:
#                    print "Tag %s is ON THE LIST" %t
                    ind_dict[ind]['read_ids'][int(t)] = []
                    add = True
            elif add and not 'model' in line:
                read = line.split("\t")[8]
#                print "FOUND READ: %s - append to %s" %(read, t)
                ind_dict[ind]['read_ids'][int(t)].append(read)
#                ind_dict[ind]['reads'][t].append(read)

        if add and int(t) in ind_catalog_tags_to_process:
            ind_catalog_tags_to_process.remove(int(t))
        print "\nDONE WITH INDIVIDUAL %s" %(ind)
        print "processed %i of %i tags\n" %((len(ind_dict[ind]['read_ids'].keys())-len(ind_catalog_tags_to_process)), len(ind_dict[ind]['read_ids'].keys()))
        if ind_catalog_tags_to_process:
            print "not processed:\n%s" %ind_catalog_tags_to_process
        ind_tags_FH.close()

                    

In [64]:
find_reads_per_individual(ind_list=ind_dict.keys(), individual_dict=ind_dict, catalog=catalog_dict)

IDs for individuals provided in list
Found data for sample_LBPNK3
Found data for sample_LBPNK2
Found data for sample_LBPNK5
Found data for sample_LBPNK4
Found data for sample_LBPNK7
Found data for sample_MONB11
Found data for sample_BBNB7
Found data for sample_BBNB6
Found data for sample_BBNB5
Found data for sample_BBNB4
Found data for sample_BBNB3
Found data for sample_BBNB2
Found data for sample_BBNB1
Found data for sample_LBPNK6
Found data for sample_BBNB9
Found data for sample_BBNB8
Found data for sample_MONB3
Found data for sample_MONB2
Found data for sample_MONB7
Found data for sample_MONB5
Found data for sample_MONB4
Found data for sample_MONB9
Found data for sample_MONB8
Found data for sample_BBNB10
Found data for sample_NJ10
Found data for sample_NJ9
Found data for sample_NJ8
Found data for sample_NJ1
Found data for sample_NJ3
Found data for sample_NJ2
Found data for sample_NJ5
Found data for sample_NJ4
Found data for sample_NJ7
Found data for sample_NJ6
Screening through 1178

In [66]:
def find_raw_reads(ind_list, individual_dict, raw_reads, pe=False):
    """
    The function checks if there are raw read files present for each sample in the list
    """
    import os.path
    import gzip
    import glob
    
#    catalog_ind_ids = {}
    
    if type(ind_list) is list: #check the value passed through ind_list is already a list
        print "IDs for individuals provided in list"
        sample_list = ind_list[:]
    else: #if it's not a list check if it's a file and parse the first column
        if not os.path.isfile(ind_list):
            IOError('ind_list needs to be either a txt file or an existing list')
        print "parsing IDs for individuals from file"
        sample_list = []
        sample_list = parse_taglist(ind_list)

    file_list = glob.glob(raw_reads+'/*fastq*')
    file_list = sorted(file_list)
    if len(file_list) == 0:
        IOError('no fastq files detected in the directory that you have specified')

#    print "Files present\n"
#    print "\n".join(file_list)
    
    for ind in sorted(sample_list):
        print "%s - checking for presence of reads in %s" %(ind, raw_reads)
    
        found_fastq_files = []
        if file_list:
            for fastq_file in file_list:
#                print "testing %s" %fastq_file
                if ind in fastq_file:
#                    print "found: %s" %fastq_file
                    found_fastq_files.append(os.path.abspath(fastq_file))
                    if not pe and len(found_fastq_files) == 1:
                        break
                    if pe and len(found_fastq_files) == 2:
                        break
        
        if not found_fastq_files:
            raise IOError("did not find any fastq file for sample %s" %sample)
        else:
            print "Found the following files for %s\n%s" %(ind, found_fastq_files)
            individual_dict[ind]['fastq_files'] = found_fastq_files
        


In [67]:
find_raw_reads(ind_list=ind_dict.keys(), individual_dict=ind_dict, raw_reads="../raw_reads/", pe=True)

IDs for individuals provided in list
sample_BBNB1 - checking for presence of reads in ../raw_reads/
Found the following files for sample_BBNB1
['/media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/raw_reads/sample_BBNB1.1.fastq.gz', '/media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/raw_reads/sample_BBNB1.2.fastq.gz']
sample_BBNB10 - checking for presence of reads in ../raw_reads/
Found the following files for sample_BBNB10
['/media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/raw_reads/sample_BBNB10.1.fastq.gz', '/media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/raw_reads/sample_BBNB10.2.fastq.gz']
sample_BBNB2 - checking for presence of reads in ../raw_reads/
Found the following files for sample_BBNB2
['/media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/raw_reads/sample_BBNB2.1.fastq.gz', '/media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/raw_reads/samp

In [69]:
def extract_reads(ind_list, individual_dict, catalog, out_dir="./", catalog_tag_whitelist=[], pe=False, per_tag=False):
    """
    The function extracts reads for each individual
    into either one file or one file each per tag.
    Optional it extracts the paired end reads as well"""
    
    import os.path
    import gzip
    from Bio import SeqIO
    
    catalog_ind_ids = {}
    
    if type(ind_list) is list: #check the value passed through ind_list is already a list
        print "IDs for individuals provided in list"
        sample_list = ind_list[:]
    else: #if it's not a list check if it's a file and parse the first column
        if not os.path.isfile(ind_list):
            IOError('ind_list needs to be either a txt file or an existing list')
        print "parsing IDs for individuals from file"
        sample_list = []
        sample_list = parse_taglist(ind_list)

    if not catalog_tag_whitelist:
        catalog_tag_whitelist = [int(i) for i in sorted(catalog.keys()[:])]
    
    for ind in sample_list:
        if not individual_dict.has_key(ind):
            raise IOError('no data found for %s' %ind)
        else:
            if not individual_dict[ind]['fastq_files']:
                raise IOError('no fastq files for %s' %s)
            catalog_ind_ids[ind_dict[ind]['catalog_id']] = ind

    sample_list = sorted(sample_list)
    
    print "Fetching reads for %i tags" %len(catalog_tag_whitelist)
    
    for ind in sample_list:
        print "#################\n"
        print ind
#        print ind_dict[ind]['tags']
#        print sorted(ind_dict[ind]['tags'])
        
        if not pe and len(individual_dict[ind]['fastq_files']) <= 2:
            fq_files = [individual_dict[ind]['fastq_files'][0]]
        elif pe and len(individual_dict[ind]['fastq_files']) == 2:
            fq_files = individual_dict[ind]['fastq_files'][:]
        else:
            raise IOError('something wrong with the fastq files for %s' %ind)
        

        for i in range(len(fq_files)):
            the_file = fq_files[i]
            file_id = i+1
            print "processing file %i: %s\n" %(file_id, the_file)
                        
            #open file
#            the_file = individual_dict[ind]['fastq_files'][0]
            if the_file.endswith('.gz'):
                reads_FH = gzip.open(the_file, 'rb')
            else:
                reads_FH = open(the_file, 'r')
        
            ind_catalog_tags_to_process = {}
            files_to_write = []
            reads_to_process = {}
            
            for catalog_tag in catalog_tag_whitelist:
                if ind_dict[ind]['tags'].has_key(catalog_tag):
                    if not per_tag:
                        ind_catalog_tags_to_process[int(ind_dict[ind]['tags'][catalog_tag])] = [os.path.abspath(out_dir)+'/'+ind+'/'+ind+'.'+str(file_id)+'.fastq'] #.append(ind_dict[ind]['tags'][catalog_tag])
                        if not os.path.exists(out_dir+"/"+ind):
                            os.makedirs(out_dir+"/"+ind)
                        if not ind_catalog_tags_to_process[int(ind_dict[ind]['tags'][catalog_tag])] in files_to_write:
                            open(ind_catalog_tags_to_process[int(ind_dict[ind]['tags'][catalog_tag])][0], 'w').close()
        
            if not ind_catalog_tags_to_process: #if this individual contains any relevant tags
                print "nothing to process for this indidividual"
                continue #go to next individual
            
            print "%s individual tags to process\n" %len(ind_catalog_tags_to_process)
            if not os.path.exists(out_dir+"/"+ind):
                os.makedirs(out_dir+"/"+ind)

            for tag in individual_dict[ind]['read_ids'].keys(): #the keys here are the individual tag ids
                for read in individual_dict[ind]['read_ids'][tag]: #the list accessed via the individual tag are the read ids
                    reads_to_process[read[:-1]+str(file_id)] = int(tag)

            complete = len(reads_to_process.keys())
            print ("total of %i reads" %complete),
        
        

            for record in SeqIO.parse(reads_FH, 'fastq'):
#                print record.id

                if reads_to_process.has_key(record.id):        
                    ind_catalog_tags_to_process[int(reads_to_process[record.id])].append(record)

                
                    if (len(ind_catalog_tags_to_process[int(reads_to_process[record.id])])-1) == len(individual_dict[ind]['read_ids'][int(reads_to_process[record.id])]):
                        OUT = open(ind_catalog_tags_to_process[int(reads_to_process[record.id])][0], 'a')
                        SeqIO.write(ind_catalog_tags_to_process[int(reads_to_process[record.id])][1:], OUT, "fastq")
                        OUT.close()
                        del ind_catalog_tags_to_process[int(reads_to_process[record.id])]

                    del reads_to_process[record.id]

                    
                if not len(reads_to_process) > 0:
                    break

            reads_FH.close()
            if len(reads_to_process) == 0:
                print " - done!\n"
            else:
                print "did not find the expected number of %i reads\n" %complete


In [70]:
import time
print time.asctime( time.localtime(time.time()) )
extract_reads(ind_list=ind_dict.keys(), individual_dict=ind_dict, catalog=catalog_dict, out_dir="./reads/", pe=True)
print time.asctime( time.localtime(time.time()) )

Tue Feb 16 14:35:18 2016
IDs for individuals provided in list
Fetching reads for 11786 tags
#################

sample_BBNB1
processing file 1: /media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/raw_reads/sample_BBNB1.1.fastq.gz

11747 individual tags to process

total of 318498 reads  - done!

processing file 2: /media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/raw_reads/sample_BBNB1.2.fastq.gz

11747 individual tags to process

total of 318498 reads  - done!

#################

sample_BBNB10
processing file 1: /media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/raw_reads/sample_BBNB10.1.fastq.gz

11722 individual tags to process

total of 243146 reads  - done!

processing file 2: /media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/raw_reads/sample_BBNB10.2.fastq.gz

11722 individual tags to process

total of 243146 reads  - done!

#################

sample_BBNB2
processing file 1: /media/chrishah/ST

run BWA

In [75]:
!mkdir ./mapping

In [76]:
%%bash

working_dir=/media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/test/mapping
data_dir=/media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/test/reads
reference=/media/chrishah/STORAGE/DATA/Cichlids/reference_data/M_zebra/MetZeb1.1_prescreen/M_zebra_v0.assembly.fasta
#prefix=M_zebra_v1.1
index=M_zebra_v1.1
MM=8
threads=2

## Do some work:
cd $working_dir

#preparing
mkdir index/; mkdir sam/; mkdir bam/

##builing indexfile
echo -e "Creating index\n"
cd index
cp -v $reference $index.fasta
bwa index -p $index -a is $index.fasta
cd ..
        
date

for sample in $(ls -1 $data_dir)
do 
        echo -e "\nprocessing sample $sample\n"
        
        bwa aln -n $MM -t $threads ./index/$index $data_dir/$sample/$sample.1.fastq > forward.sai
#        bwa aln -t $threads ./index/$index $data_dir/$sample/$sample.2.fastq > reverse.sai
        RG="@RG\tID:$sample\tSM:$sample"
        bwa samse -r $RG ./index/$index forward.sai $data_dir/$sample/$sample.1.fastq > sam/$sample.sam
#        bwa sampe -r $RG $index_pre forward.sai reverse.sai data/$prefix\_1.fastq data/$prefix\_2.fastq > sam/$prefix.sam
        rm -v forward.sai reverse.sai 

        cd bam/
        samtools view -@ $threads -bS -o $sample.bam ../sam/$sample.sam
        samtools sort -@ $threads $sample.bam $sample.sorted
        samtools index $sample.sorted.bam
#        samtools idxstats $sample.sorted.bam
        rm -v $sample.bam
        cd ../
#        gzip -v sam/*.sam
#        rm -rfv ./sam
        date
done


Creating index

‘/media/chrishah/STORAGE/DATA/Cichlids/reference_data/M_zebra/MetZeb1.1_prescreen/M_zebra_v0.assembly.fasta’ -> ‘M_zebra_v1.1.fasta’
Tue Feb 16 20:30:50 GMT 2016

processing sample sample_BBNB1

removed ‘forward.sai’
removed ‘sample_BBNB1.bam’
Tue Feb 16 20:31:25 GMT 2016

processing sample sample_BBNB10

removed ‘forward.sai’
removed ‘sample_BBNB10.bam’
Tue Feb 16 20:31:49 GMT 2016

processing sample sample_BBNB2

removed ‘forward.sai’
removed ‘sample_BBNB2.bam’
Tue Feb 16 20:32:05 GMT 2016

processing sample sample_BBNB3

removed ‘forward.sai’
removed ‘sample_BBNB3.bam’
Tue Feb 16 20:33:01 GMT 2016

processing sample sample_BBNB4

removed ‘forward.sai’
removed ‘sample_BBNB4.bam’
Tue Feb 16 20:33:38 GMT 2016

processing sample sample_BBNB5

removed ‘forward.sai’
removed ‘sample_BBNB5.bam’
Tue Feb 16 20:34:26 GMT 2016

processing sample sample_BBNB6

removed ‘forward.sai’
removed ‘sample_BBNB6.bam’
Tue Feb 16 20:34:53 GMT 2016

processing sample sample_BBNB7

removed ‘f

[bwa_index] Pack FASTA... 8.41 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 551.97 seconds elapse.
[bwa_index] Update BWT... 5.03 sec
[bwa_index] Pack forward-only FASTA... 5.71 sec
[bwa_index] Construct SA from BWT and Occ... 205.17 sec
[main] Version: 0.7.10-r789
[main] CMD: bwa index -p M_zebra_v1.1 -a is M_zebra_v1.1.fasta
[main] Real time: 794.800 sec; CPU: 776.298 sec
[bwa_aln_core] calculate SA coordinate... 40.03 sec
[bwa_aln_core] write to the disk... 0.03 sec
[bwa_aln_core] 262144 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 10.84 sec
[bwa_aln_core] write to the disk... 0.01 sec
[bwa_aln_core] 318498 sequences have been processed.
[main] Version: 0.7.10-r789
[main] CMD: bwa aln -n 8 -t 2 ./index/M_zebra_v1.1 /media/chrishah/STORAGE/RAD/popgen/Freebayes/Diplotaxodon_4pop_for_paper/test/reads/sample_BBNB1/sample_BBNB1.1.fastq
[main] Real time: 26.662 sec; CPU: 51.564 sec
[bwa_aln_core] convert to sequence coordinate... 0.93 se

Do SNPcalling

In [77]:
!mkdir ./SNP-calling/

In [7]:
%%bash

cd SNP-calling/
for sample in $(cut -f 1 ../../popmap); do ls -1 ../mapping/bam/$sample.* |grep "bam$"; done > bamlist.txt

freebayes -L bamlist.txt -f ../mapping/index/M_zebra_v1.1.fasta -v full.vcf --report-monomorphic --populations ../../popmap --no-complex
cd ..

extract fasta inclusive reference

In [10]:
%%bash

/media/chrishah/STORAGE/Dropbox/Github/genomisc/popogeno/vcf_2_consensus.py SNP-calling/full.vcf -p ../popmap -o SNP-calling/full 


ignoring indels: scaffold_0 937795
ignoring indels: scaffold_0 959661
ignoring > bi-allelic SNPs: scaffold_0 973178
ignoring > bi-allelic SNPs: scaffold_0 1985535
ignoring indels: scaffold_0 2290805
ignoring > bi-allelic SNPs: scaffold_0 2339302
ignoring indels: scaffold_0 2558792
ignoring indels: scaffold_0 3566065
ignoring indels: scaffold_0 4883311
ignoring > bi-allelic SNPs: scaffold_0 5562771
ignoring indels: scaffold_0 5570042
ignoring indels: scaffold_0 6248672
ignoring indels: scaffold_0 6283735
ignoring indels: scaffold_0 6689464
ignoring indels: scaffold_0 6723865
ignoring > bi-allelic SNPs: scaffold_0 6844277
ignoring indels: scaffold_0 6844319
ignoring indels: scaffold_0 7079686
ignoring indels: scaffold_0 7591476
ignoring indels: scaffold_0 8203013
ignoring indels: scaffold_0 8281824
ignoring indels: scaffold_0 9069215
ignoring indels: scaffold_0 9280032
ignoring indels: scaffold_0 10060904
ignoring indels: scaffold_0 11143410
ignoring > bi-allelic SNPs: scaffold_0 1203282

run RAXML without outgroup

In [11]:
%%bash

mkdir RAXML-unrooted
cd RAXML-unrooted
/home/chrishah/src/RAxML/RAxML-8.2.4/raxmlHPC-PTHREADS-SSE3 -f a -T 3 -x 12345 -p 12345 -N 100 -m GTRGAMMA -s ../SNP-calling/full.fas -n full
cd ..


RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file

IMPORTANT WARNING: Alignment column 185231 contains only undetermined values which will be treated as missing data
IMPORTANT WARNING: Alignment column 347399 contains only undetermined values which will be treated as missing data
IMPORTANT WARNING: Alignment column 532781 contains only undetermined values which will be treated as missing data
IMPORTANT WARNING: Alignment column 552907 contains only undetermined values which will be treated as missing data

IMPORTANT WARNING
Found 4 columns that contain only undetermined values which will be treated as missing data.
Normally these columns should be excluded from the analysis.

Just in case you might need it, an alignment file with 
undetermined columns removed is printed to file ../SNP-calling/full.fas.reduced


Using BFGS method to optimize GTR rate parameters, to disable this specify "--no-bfgs" 


Alignment has 4 completely undetermined 